In [137]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
import pandas as pd
import re

# 事前準備
url = 'https://suumo.jp/chintai/tokyo/sc_koto/?page={}'
d_list = []

for i in range(1,10):
    target_url = url.format(i)
    r = requests.get(target_url)
    
    # 1秒ずつ取得
    time.sleep(1)
    soup = BeautifulSoup(r.text,"html.parser")
    
    contents = soup.find_all('div', class_='cassetteitem')

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')
        
        # タイトルを取得し、空白を削除
        title_text = detail.find('div', class_='cassetteitem_content-title').text
        title = title_text.replace('　', '')
        
        # アドレスを取得
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        
        # アクセスを取得、改行を削除、要素間をカンマ区切りに
        access_text = detail.find('li', class_='cassetteitem_detail-col2').text
        cleaned_access_text = access_text.strip()
        access = cleaned_access_text.replace('\n', ',')
        
        # 築年数と総階数を取得、分割してageとstoryに格納。地下は重要度低いため除外
        spaced_age_and_story = detail.find('li', class_='cassetteitem_detail-col3').text
        age_and_story = spaced_age_and_story = spaced_age_and_story.replace('築0年', '築1年').replace('新築', '築1年')
        numbers = re.findall(r'\d+', age_and_story)
        if len(numbers) == 2:
            age = int(numbers[0])
            story = int(numbers[1])
        elif len(numbers) == 3:
            age = int(numbers[0])
            story = int(numbers[2])
        else:
            age = 0
            story = 0
            
        # trタグから部屋情報を取得
        trtags = table.find_all('tr', class_='js-cassette_link')
        for trtag in trtags:
            original_floor_data, price, first_fee, capacity = trtag.find_all('td')[2:6]
            
            # 階数を取得、複数階は最低階を格納、階なしは1を格納
            floor_text = original_floor_data.text
            matched_floor_number = re.search(r'\d+', floor_text)
            if matched_floor_number:
                floor = int(matched_floor_number.group())
            else:
                floor = 1
            
            # strを引数、数字のみ抽出したfloat型のnumberを返す関数
            def extract_number(text):
                matched_text = re.search(r'\d+(\.\d+)?', text)
                if matched_text:
                    number = float(matched_text.group())
                else:
                    number = 0
                return number
            
            # 各オブジェクトに関数適用してfloat型に揃える
            fee_text, management_fee_text = [li.text for li in price.find_all('li')]
            fee = extract_number(fee_text)
            management_fee = extract_number(management_fee_text)/10000 # 管理費も万円に揃える
            
            deposit_text, gratuity_text = [li.text for li in first_fee.find_all('li')]
            deposit = extract_number(deposit_text)
            gratuity = extract_number(gratuity_text)
            
            madori, menseki_text = [li.text for li in capacity.find_all('li')]
            menseki = extract_number(menseki_text)
            
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'story': story,
                'floor': floor,
                'fee': fee,
                'management_fee': management_fee,
                'deposit': deposit,
                'gratuity': gratuity,
                'madori': madori,
                'menseki': menseki,
            }
            d_list.append(d)

df = pd.DataFrame(d_list)

In [123]:
num = df.head()
num

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,プラウドシティ越中島ブライトテラス,東京都江東区越中島３,"ＪＲ京葉線/越中島駅 歩6分,東京メトロ東西線/門前仲町駅 歩12分,東京メトロ有楽町線/豊...",5,12,4,25.0,0.0,50.0,25.0,3SLDK,72.87
1,ルカナル門前仲町,東京都江東区富岡２,"東京メトロ東西線/門前仲町駅 歩7分,東京メトロ東西線/木場駅 歩6分,ＪＲ京葉線/越中島駅...",4,10,2,11.6,0.7,11.6,11.6,1DK,30.26
2,ルカナル門前仲町,東京都江東区富岡２,"東京メトロ東西線/門前仲町駅 歩7分,東京メトロ東西線/木場駅 歩6分,ＪＲ京葉線/越中島駅...",4,10,4,17.9,1.0,17.9,35.8,2LDK,50.73
3,東京メトロ東西線 木場駅 10階建 築4年,東京都江東区富岡２,"東京メトロ東西線/木場駅 歩6分,都営大江戸線/門前仲町駅 歩7分,ＪＲ京葉線/越中島駅 歩15分",4,10,2,11.6,0.7,11.6,11.6,1DK,30.26
4,東京メトロ東西線 木場駅 10階建 築4年,東京都江東区富岡２,"東京メトロ東西線/木場駅 歩6分,都営大江戸線/門前仲町駅 歩7分,ＪＲ京葉線/越中島駅 歩15分",4,10,4,17.9,1.0,17.9,35.8,2LDK,50.73


In [126]:
len(df)

734

In [127]:
df_c = df.drop_duplicates()
len(df_c)

685

In [134]:
df_1 = df.drop_duplicates(['address', 'age', 'story', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'madori', 'menseki'])
filtered_df = df_1.groupby(['address', 'age', 'story', 'floor', 'fee', 'deposit', 'gratuity', 'madori', 'menseki']).filter(lambda x: len(x) > 1)
filtered_df

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,madori,menseki
47,BLESS亀戸,東京都江東区大島３,"ＪＲ総武線/亀戸駅 歩7分,都営新宿線/西大島駅 歩6分,東京メトロ半蔵門線/錦糸町駅 歩18分",1,19,7,20.7,1.5,20.7,20.7,2LDK,51.90
51,ＪＲ総武線 亀戸駅 19階建 築1年,東京都江東区大島３,"ＪＲ総武線/亀戸駅 歩7分,都営新宿線/西大島駅 歩6分,東京メトロ半蔵門線/錦糸町駅 歩18分",1,19,7,20.7,2.0,20.7,20.7,2LDK,51.90
164,東雲キャナルコートCODAN18号棟,東京都江東区東雲１,"東京メトロ有楽町線/辰巳駅 歩9分,りんかい線/東雲駅 歩12分,新交通ゆりかもめ/豊洲駅 ...",20,14,7,15.4,1.2,15.4,15.4,1LDK,53.13
166,東雲キャナルコートCODAN18号棟,東京都江東区東雲１,"東京メトロ有楽町線/辰巳駅 歩9分,りんかい線/東雲駅 歩12分,新交通ゆりかもめ/豊洲駅 ...",20,14,7,15.4,1.5,15.4,15.4,1LDK,53.13
177,東京メトロ東西線 木場駅 7階建 築8年,東京都江東区東陽５,"東京メトロ東西線/木場駅 歩7分,東京メトロ東西線/東陽町駅 歩7分,都営大江戸線/門前仲町...",8,7,3,8.9,0.5,8.9,8.9,1K,25.62
178,東京メトロ東西線 木場駅 7階建 築8年,東京都江東区東陽５,"東京メトロ東西線/木場駅 歩7分,東京メトロ東西線/東陽町駅 歩7分,都営大江戸線/門前仲町...",8,7,3,8.9,0.8,8.9,8.9,1K,25.62
225,パークアクシス豊洲,東京都江東区豊洲１,"東京メトロ有楽町線/豊洲駅 歩8分,都営大江戸線/月島駅 歩14分,ＪＲ京葉線/越中島駅 歩15分",16,20,12,20.2,0.6,20.2,20.2,1LDK,47.31
226,パークアクシス豊洲,東京都江東区豊洲１,"東京メトロ有楽町線/豊洲駅 歩8分,都営大江戸線/月島駅 歩14分,ＪＲ京葉線/越中島駅 歩15分",16,20,12,20.2,1.0,20.2,20.2,1LDK,47.31
231,パークアクシス豊洲,東京都江東区豊洲１,"東京メトロ有楽町線/豊洲駅 歩8分,都営大江戸線/月島駅 歩14分,ＪＲ京葉線/越中島駅 歩15分",16,20,5,35.6,1.0,35.6,35.6,3LDK,80.82
232,パークアクシス豊洲,東京都江東区豊洲１,"東京メトロ有楽町線/豊洲駅 歩8分,都営大江戸線/月島駅 歩14分,ＪＲ京葉線/越中島駅 歩15分",16,20,5,35.6,1.2,35.6,35.6,3LDK,80.82


In [132]:
df_2 = df.drop_duplicates(['address', 'age', 'story', 'floor', 'fee', 'deposit', 'gratuity', 'madori', 'menseki'])
len(df_2)

576

In [138]:
df_3 = df[df['menseki'] == 51.90]
df_3

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,madori,menseki
419,BLESS亀戸,東京都江東区大島３,"都営新宿線/西大島駅 歩7分,東武亀戸線/亀戸駅 歩7分,都営新宿線/大島駅 歩15分",1,19,7,20.7,1.5,20.7,20.7,2LDK,51.9
466,ＪＲ総武線 亀戸駅 19階建 築1年,東京都江東区大島３,"ＪＲ総武線/亀戸駅 歩7分,都営新宿線/西大島駅 歩6分,東京メトロ半蔵門線/錦糸町駅 歩18分",1,19,7,20.7,1.5,20.7,20.7,2LDK,51.9
467,ＪＲ総武線 亀戸駅 19階建 築1年,東京都江東区大島３,"ＪＲ総武線/亀戸駅 歩7分,都営新宿線/西大島駅 歩6分,東京メトロ半蔵門線/錦糸町駅 歩18分",1,19,7,20.7,2.0,20.7,20.7,2LDK,51.9


In [141]:
df_dropped = df.drop_duplicates(['address', 'age', 'story', 'floor', 'madori', 'menseki'])
len(df_dropped)

524

In [142]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [163]:
# 認証情報の設定
SP_CREDENTIAL_FILE = '/Users/ryosukeinoue/Documents/31_Tech0/Step3/scraping-techone-8842efd0d979.json'
SP_SCOPE = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/spreadsheets'
]
SP_SHEET_KEY = '1wi_aezq9wK88OtUprgOE40R4AlHGEoVUYmT31HbpzCo'
credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_SCOPE)

In [164]:
# gspreadクライアントの初期化
gc = gspread.authorize(credentials)

In [165]:
# スプレッドシートを開き、最初のワークシートを選択
spreadsheet = gc.open_by_key(SP_SHEET_KEY)
worksheet = spreadsheet.sheet1

In [166]:
# スプレッドシートへのヘッダーの書き込み
worksheet.append_row(df.columns.tolist())

{'spreadsheetId': '1wi_aezq9wK88OtUprgOE40R4AlHGEoVUYmT31HbpzCo',
 'updates': {'spreadsheetId': '1wi_aezq9wK88OtUprgOE40R4AlHGEoVUYmT31HbpzCo',
  'updatedRange': "'シート1'!A1:L1",
  'updatedRows': 1,
  'updatedColumns': 12,
  'updatedCells': 12}}

In [169]:
# DataFrameのデータを2Dリストに変換
data = df.values.tolist()

worksheet.update('A2', data)

/var/folders/t6/4471p9392z319n7yg_mhfvvh0000gn/T/ipykernel_50868/3701130758.py:4: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update('A2', data)


{'spreadsheetId': '1wi_aezq9wK88OtUprgOE40R4AlHGEoVUYmT31HbpzCo',
 'updatedRange': "'シート1'!A2:L525",
 'updatedRows': 524,
 'updatedColumns': 12,
 'updatedCells': 6288}